# Discrete Element Method for Rigid Body Simulation

For this project, I have implemented a method of simulating rigid body dynamics using the discrete element approximation, which treats each rigid body as being composed of discrete particles. While this method has some drawbacks such as dependence on the sampling resolution of the particles for the quality of results, it is extremely well suited for parallel implementation on the GPU. It is also agnostic to model topology, unlike some analytic methods which work best on convex models, for instance.

## Running this notebook
Place the `.txt` files in the same directory as this notebook before running it.
### Dependencies
Along with the obvious `cuda-toolkit` and numba dependencies, this notebook depends on the libraries `igl` and `meshplot`, which can be installed via anaconda.

## Overview

The pipeline is divided into the following stages:
- Sample points in the volume of each mesh with uniform density (using fast winding numbers to determine the interior of meshes, allowing non-watertight input meshes)
- Precompute various invariant properties of each rigid body, such as the mass and moment of inertia tensor
- Allocate a grid data structure to keep track of the spatial locations of each object's particles
- For each time step:
 - Repopulate the data structure using a CUDA scattering kernel
 - Compute the forces on each particle, along with resulting torques at each point, by searching the neighboring grid cells for nearby particles (also in a CUDA kernel)
 - Using CUDA reduction, compute the net force and torque on each rigid body as a whole
 - Update the state by integrating the laws of motion
 
## Inter-particle forces
By the rigid body assumption, the net force due to other particles in the same object is zero. So for all particles in different objects, we define the following forces:

$$\mathbf f_{i,s}=-k(d-|\mathbf r_{ij}|)\frac{\mathbf r_{ij}}{|\mathbf r_{ij}|}$$

is the spring force that acts when particles $i$ and $j$ are less than distance $d$ apart, and

$$\mathbf f_{i,d}=\eta \mathbf v_{ij}$$

is the damping force (where $\mathbf v_{ij}$ is the relative velocity of the two particles).

## Updating state
We use the laws of motion for rigid body mechanics. Angular momentum and momentum are affected by torque and force, respectively:
$$\dot L = \mathbf \tau$$
$$\dot p = \mathbf F$$
where $\tau=\sum_i (r_i-c)\times F_i$ and $c$ is the center of mass.
Velocity is related to momentum by the simple formula $v=p/M$, but rotations are somewhat trickier. Angular velocity $\omega$ is related to angular momentum via the *inertia tensor* $I$, so that $$L = I\omega$$. $I$ also varies in time, but it turns out that a time-invariant matrix can be precomputed in the rigid body's rest coordinate frame--we store the inverse of this matrix to quickly compute the angular velocity on demand. For more info on the details, see https://www.cs.cmu.edu/~baraff/sigcourse/notesd1.pdf.


## Viewing results
For now, the closest thing to displaying 3D animations in these notebooks that I have found is to have interactive plots, with a slider controlling the frame under examination.

In [1]:
from meshplot import plot, interact
import ipywidgets
import numpy as np
import numpy.linalg as la
import igl
import time, math
import os

In [2]:
from numba import cuda, jit, prange, njit, int32, float32, void

## Utility functions for precomputing samples, and integrating mass and moment of inertia

In [3]:
def voxel_points(axes):
    """generate n-dimensional grid coordinates from the given set of (x, y, z...) values"""
    ndims = len(axes)
    dims = [len(x) for x in axes]
    indices = np.indices(dims)
    points = np.array([axis[index] for axis, index in zip(axes, indices)])
    points = points.reshape([ndims, np.prod(dims)])
    return points.T.copy()

In [4]:
def sample_mesh_interior(V, F, gridLength, dtype=np.float32):
    minPt = np.min(V, 0)
    maxPt = np.max(V, 0)
    q = voxel_points((np.arange(minPt[0], maxPt[0] + gridLength, gridLength, dtype=dtype),
                       np.arange(minPt[1], maxPt[1] + gridLength, gridLength, dtype=dtype),
                         np.arange(minPt[2], maxPt[2] + gridLength, gridLength, dtype=dtype)))
    #print(q)
    w = igl.fast_winding_number_for_meshes(V.astype(dtype), F, q)
    #print(w.shape)
    return q[w > 0.5]

@njit (parallel=True)
def compute_inertia_tensor(points, pointmass):
    inertia_tensor = np.zeros((3, 3), dtype=points.dtype)
    N = points.shape[0]
    for i in prange(N):
        pointCM = points[i,:]
        moments = pointCM * pointCM
        inertia_tensor[0, 0] += moments[1] + moments[2]
        inertia_tensor[1, 1] += moments[0] + moments[2]
        inertia_tensor[2, 2] += moments[0] + moments[1]
        inertia_tensor[0, 1] = pointCM[0] * pointCM[1]
        inertia_tensor[0, 2] = pointCM[0] * pointCM[2]
        inertia_tensor[1, 2] = pointCM[1] * pointCM[2]
    inertia_tensor *= pointmass
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]
    return inertia_tensor

In [5]:
class PhysObject:
    def __init__(self, V, F, gridDim, density=1):
        self.points = sample_mesh_interior(V, F, gridDim)
        pointmass = density * gridDim ** 3
        cm = np.mean(self.points, 0)
        self.points -= cm
        self.mass = pointmass * self.points.shape[0]
        self.inertia_tensor = compute_inertia_tensor(self.points, pointmass)
        self.inertia_tensor_inv = la.inv(self.inertia_tensor)

## Vector math prerequisites

In [6]:
@njit
def quaternion_to_matrix(q):
    s = q[0]
    vx = q[1]
    vy = q[2]
    vz = q[3]
    vx2 = vx*vx
    vy2 = vy*vy
    vz2 = vz*vz
    return np.array([[1-2*vy2-2*vz2, 2*vx*vy-2*s*vz, 2*vx*vz+2*s*vy],
                     [2*vx*vy+2*s*vz, 1-2*vx2-2*vz2, 2*vy*vz-2*s*vx],
                     [2*vx*vz-2*s*vy, 2*vy*vz+2*s*vx, 1-2*vx2-2*vy2]])

@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def hamilton_product_device(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3

@njit
def hamilton_product_host(p, q, out):
    out0 = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    out1 = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    out2 = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    out3 = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    out[3] = out3


@cuda.jit(void(float32[:], float32[:], float32[:]), device=True)
def cross_product_device(p, q, out):
    a0 = p[1] * q[2] - p[2] * q[1]
    a1 = p[2] * q[0] - p[0] * q[2]
    a2 = p[0] * q[1] - p[1] * q[0]
    out[0] = a0
    out[1] = a1
    out[2] = a2

@njit
def cross_product_host(p, q, out):
    out0 = p[1] * q[2] - p[2] * q[1]
    out1 = p[2] * q[0] - p[0] * q[2]
    out2 = p[0] * q[1] - p[1] * q[0]
    out[0] = out0
    out[1] = out1
    out[2] = out2
    
@cuda.jit(device=True)
def transform_device(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_device(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_device(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

@njit
def transform_host(p, pose, out):
    """transform the point p by the pose [qw, qx, qy, qz, tx, ty, tz], where p and out are both represented quaternions with no scalar part"""
    #rotate
    hamilton_product_host(pose, p, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    hamilton_product_host(out, pose, out)
    pose[1] = -pose[1]
    pose[2] = -pose[2]
    pose[3] = -pose[3]
    #translate
    out[1] += pose[4]
    out[2] += pose[5]
    out[3] += pose[6]

## CUDA-based implementation of rigid body simulation

In [7]:
cell_size = 2
    
@cuda.jit
def populate_grid_kernel(points1, pose1, points2, pose2, grid, minPt, gridDim):
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(7, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(7):
            pose[j] = pose1[j]
        offset = 0
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(7):
            pose[j] = pose2[j]
        offset = cell_size
    else:
        return
    
    transform_device(transpoint, pose, transpoint)
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        for j in range(cell_size):
            cellval = cuda.atomic.compare_and_swap(grid[px, py, pz*2*cell_size+offset+j:], -1, i)
            if cellval == -1:
                break
            #if grid[px, py, pz * 2 * cell_size + offset + j] < 0:
                #grid[px, py, pz * 2 * cell_size + offset + j] = i
            #    break
                
@cuda.jit
def empty_grid(grid):
    i, j, k = cuda.grid(3)
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2]
    if i < nx and j < ny and k < nz:
        grid[i, j, k] = -1
                
@cuda.jit
def compute_forces(outForces1, points1, pose1, outForces2, points2, pose2, grid, minPt, gridDim, springK, damping, debug_points1, debug_points2):
    radius = gridDim # maximum distance between interacting particles
    radius2 = radius * radius
    n1 = points1.shape[0]
    n2 = points2.shape[0]
    i = cuda.grid(1)
    transpoint = cuda.local.array(4, float32)
    pose = cuda.local.array(13, float32)
    otherpose = cuda.local.array(13, float32)
    transpoint[0] = 0
    if i<n1:
        for j in range(3):
            transpoint[j+1] = points1[i,j]
        for j in range(13):
            pose[j] = pose1[j]
            otherpose[j] = pose2[j]
        offset = 0
        otherOffset = cell_size
        otherpoints = points2
        outForces = outForces1
        debug_arr = debug_points1
    elif i<n1 + n2:
        i -= n1
        for j in range(3):
            transpoint[j+1] = points2[i,j]
        for j in range(13):
            pose[j] = pose2[j]
            otherpose[j] = pose1[j]
        offset = cell_size
        otherOffset = 0
        otherpoints = points1
        outForces = outForces2
        debug_arr = debug_points2
    else:
        return
    
    transform_device(transpoint, pose, transpoint)
    point_cm = cuda.local.array(3, float32)
    for k in range(3):
        point_cm[k] = transpoint[k+1] - pose[k+4]
    #print('transpoint:',transpoint[1], transpoint[2], transpoint[3])
    px = int(math.floor((transpoint[1] - minPt[0])/gridDim))
    py = int(math.floor((transpoint[2] - minPt[1])/gridDim))
    pz = int(math.floor((transpoint[3] - minPt[2])/gridDim))
    nx = grid.shape[0]
    ny = grid.shape[1]
    nz = grid.shape[2] // (cell_size*2)
    if px >= 0 and px < nx and py >= 0 and py < ny and pz >= 0 and pz < nz:
        #print('cell:',px, py, pz)
        for j in range(cell_size):
            found = False
            #if grid[px, py, pz * 2 * cell_size + offset + j] >= 0:
                #print('indices:',i, grid[px, py, pz * 2 * cell_size + offset + j])
            if grid[px, py, pz * 2 * cell_size + offset + j] == i:
                found = True
                break
        if not found:
            return
        force = cuda.local.array(3, float32)
        tempforce = cuda.local.array(3, float32)
        torque = cuda.local.array(3, float32)
        temptorque = cuda.local.array(3, float32)
        omega = cuda.local.array(3, float32)
        otheromega = cuda.local.array(3, float32)
        for l in range(3):
            force[l] = 0
            torque[l] = 0
            omega[l] = pose[l+7]
            otheromega[l] = otherpose[l+7]
        
        #print('otheromega:',otheromega[0], otheromega[1], otheromega[2])#, 'othervel:',othervel[0], othervel[1], othervel[2])
            
        vel = cuda.local.array(3, float32)
        #print('velbefore:',vel[0], vel[1], vel[2])
        cross_product_device(omega, point_cm, vel)
        #debug
        for k in range(3):
            debug_arr[i,k] = vel[k]
        #print('omega:',omega[0], omega[1], omega[2],
        #      'point_cm:',point_cm[0], point_cm[1], point_cm[2],'vel:',vel[0], vel[1], vel[2])
        
        otherpoint = cuda.local.array(4, float32)
        otherpoint_cm = cuda.local.array(3, float32)
        othervel = cuda.local.array(3, float32)
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                for dz in range(-1, 2):
                    for j in range(cell_size):
                        ox = px+dx
                        oy = py+dy
                        oz = pz+dz
                        if ox < 0 or ox >= nx or oy < 0 or oy >= ny or oz < 0 or oz >= nz:
                            continue
                        otherIndex = grid[ox,oy,oz*2*cell_size+otherOffset+j]
                        if otherIndex >= 0:
                            otherpoint[0] = 0
                            for k in range(3):
                                otherpoint[k+1] = otherpoints[otherIndex, k]
                            transform_device(otherpoint, otherpose, otherpoint)
                            for k in range(3):
                                otherpoint_cm[k] = otherpoint[k+1] - otherpose[k+4]
                                #otherpoint holds the displacement from otherpoint to this point
                                otherpoint[k+1] = transpoint[k+1] - otherpoint[k+1]                                
                            
                            dist2 = otherpoint[1]*otherpoint[1]+otherpoint[2]*otherpoint[2]+otherpoint[3]*otherpoint[3]
                            dist = math.sqrt(dist2)
                            #print(dist)
                            #spring force
                            if dist2 > 0 and dist2 < radius2:
                                    springfac = springK * (radius - dist)/dist
                                    for k in range(3):
                                        tempforce[k] = springfac * otherpoint[k+1]
                                        #print('realvalue:',springfac * otherpoint[k+1],'got:',tempforce[k])
                                        #force[k] += tempforce[k]
                                    cross_product_device(point_cm, tempforce, temptorque)
                                    for k in range(3):
                                        force[k] += tempforce[k]
                                        torque[k] += temptorque[k]
                                        
                            #damping force
                            #compute relative velocity of points
                            cross_product_device(otheromega, otherpoint_cm, othervel)
                            for k in range(3):
                                tempforce[k] = damping * (othervel[k] - vel[k])
                            cross_product_device(point_cm, tempforce, temptorque)
                            for k in range(3):
                                force[k] += tempforce[k]
                                torque[k] += temptorque[k]
                            
        outForces[i, 0] = force[0]
        outForces[i, 1] = force[1]
        outForces[i, 2] = force[2]
        outForces[i, 3] = torque[0]
        outForces[i, 4] = torque[1]
        outForces[i, 5] = torque[2]

TPB = 32


@cuda.jit
def reduce_forces_torques(d_accum, d_f):
    i = cuda.grid(1)
    tIdx = cuda.threadIdx.x
    n = d_f.shape[0]
    sh_w = cuda.shared.array((TPB, 6), dtype=float32)
    #for j in range(6):
    #    sh_w[tIdx, j] = 0
    if i < n:
        for j in range(6):
            sh_w[tIdx, j] = d_f[i, j]
    if i < 6:
        d_accum[i] = 0
    cuda.syncthreads()
    if tIdx < 6:
        component_sum = 0.0
        for j in range(cuda.blockDim.x):
            component_sum += sh_w[j, tIdx]
        cuda.atomic.add(d_accum, tIdx, component_sum)

def simulate_rigid_bodies(v1, f1, p1, m1, v2, f2, p2, m2, gridLen, minPt, maxPt, springK, damping, dt, steps):
    print('minPt:',minPt, 'maxPt:',maxPt, 'gridLen:',gridLen)
    start = time.time()
    objects = [PhysObject(v, f, gridLen, m) for v, f, m in zip((v1, v2), (f1, f2), (m1, m2))]
    end = time.time()
    print('initailized objects in',end-start)
    
    print('points in obj1:',objects[0].points.shape[0])
    print('points in obj2:',objects[1].points.shape[0])
    
    #minPt = np.minimum(np.min(v1, 0), np.min(v2, 0))
    #maxPt = np.maximum(np.max(v1, 0), np.max(v2, 0))
    
    res = np.ceil(((maxPt - minPt) / gridLen)).astype(np.int32)
    res[2] *= cell_size * 2

    TPCB = 4
    blockX = (res[0] + TPCB - 1) // TPCB
    blockY = (res[1] + TPCB - 1) // TPCB
    blockZ = (res[2] + TPCB - 1) // TPCB
    
    
    start = time.time()
    
    grid = cuda.to_device(np.full(res, -1, dtype=np.int32))
    #quaternion as [wxyz], followed by translation [xyz]
    states = []
    states_d = []
    points_d = []
    vel_debug_d = [cuda.device_array(obj.points.shape, np.float32) for obj in objects]
    forces_d = [cuda.device_array((obj.points.shape[0], 6), np.float32) for obj in objects] #[force, torque]
    forces_accum_d = [cuda.device_array(6, np.float32) for i in range(2)]
    
    for i in range(2):
        state = np.zeros((13,), dtype=np.float32)
        state[:] = (p1, p2)[i]
        states.append(state)
        #print(state)
        states_d.append(cuda.to_device(state))
        points_d.append(cuda.to_device(objects[i].points))
        
    minPt_d = cuda.to_device(minPt)
    end = time.time()
    print('copied data to GPU in',end-start)
    
    
    m = np.sum([obj.points.shape[0] for obj in objects])
    gridDim = (m + TPB - 1) // TPB
    
    #net conserved state
    L = [st[7:10].copy() for st in states]
    p = [st[10:13].copy() for st in states]
    
    print('initial momentum:',p)
    print('initial angmom:',L)
    
    allstates = []
    
    starte = cuda.event()
    ende = cuda.event()
    
    for i in range(steps):
        print('frame',i)
        starte.record()
        empty_grid[(blockX, blockY, blockZ), (TPCB, TPCB, TPCB)](grid)
        
        #debug
        #gridtest = grid.copy_to_host()
        #if np.sum(gridtest != -1) != 0:
        #    print('NOT PROPERLY CLEARED!', gridtest[gridtest != -1])
        
        populate_grid_kernel[gridDim, TPB](points_d[0], states_d[0], points_d[1], states_d[1], grid, minPt_d, gridLen)
        ende.record()
        ende.synchronize()
        print('populated grid in',cuda.event_elapsed_time(starte, ende)/1000)
        
        starte.record()
        compute_forces[gridDim, TPB](forces_d[0], points_d[0], states_d[0], forces_d[1], points_d[1], states_d[1], grid, minPt_d, gridLen, springK, damping, vel_debug_d[0], vel_debug_d[1])
        ende.record()
        ende.synchronize()
        print('computed individual point forces in',cuda.event_elapsed_time(starte, ende)/1000)
        
        #debug
        #transpoints = [arr.copy_to_host() for arr in trans_points_debug_d]
        #for j in range(2):
        #    for k in range(objects[j].points.shape[0]):
        #        point = np.zeros(4)
        #        point[1:] = objects[j].points[k,:]
        #        transform_host(point, states[j], point)
        #        diff = transpoints[j][k,:] - point[1:]
        #        print('difference',j,k,':',diff)
        if i == 0:
            gridtest = grid.copy_to_host()
            grid1 = gridtest[:, :, ::(2*cell_size)]
            grid2 = gridtest[:, :, cell_size::(2*cell_size)]
            mask1 = grid1 >= 0
            mask2 = grid2 >= 0
            pl = plot(np.indices(grid1.shape)[:, mask1].T, c='blue')
            pl.add_points(np.indices(grid2.shape)[:, mask2].T, c='red')
            mask3 = mask1 & mask2
            if mask3.sum() > 0:
                print(mask3.sum(),'shared cells')
                pl.add_points(np.indices(grid2.shape)[:, mask3].T, c='green', shading={"point_size": 3})
            pl
        #for px in range(res[0]):
        #    for py in range(res[1]):
        #        for pz in range(res[2] // (cell_size*2)):
        #            index1 = gridtest[px, py, pz*2*cell_size]
        #            index2 = gridtest[px, py, pz*2*cell_size+cell_size]
        #            if index1 >= 0 and index2 >= 0:
        #                point1 = np.zeros(4)
        #                point2 = np.zeros(4)
        #                point1[1:] = objects[0].points[index1,:]
        #                point2[1:] = objects[1].points[index2,:]
        #                transform_host(point1, states[0], point1)
        #                transform_host(point2, states[1], point2)
        #                #print('points:',point1, point2)
        #                #point1 += states[0][4:7]
        #                #point2 += states[1][4:7]
        #                diff = point2[1:] - point1[1:]
        #                print('distance',px,py,pz,'between',index1,index2,':',math.sqrt(diff.dot(diff)))
        #end debug
        
        for j in range(2):
            gridDimi = (objects[j].points.shape[0] + TPB - 1) // TPB
            starte.record()
            reduce_forces_torques[gridDimi, TPB](forces_accum_d[j], forces_d[j])
            ende.record()
            ende.synchronize()
            print('reduced forces',j,'on gpu in',cuda.event_elapsed_time(starte, ende)/1000)
            forces_accum = forces_accum_d[j].copy_to_host()
            
            print('force ',j,':',forces_accum[:3])
            print('torque ',j,':',forces_accum[3:])
            
            #debug
            #allforces = forces_d[j].copy_to_host()
            #start = time.time()
            #cpu_force = np.sum(allforces, 0)
            #end = time.time()
            #print('reduced forces',j,'on cpu in',end-start)
            #print('force_gpu:',forces_accum,'force_cpu:',cpu_force)
                        
            p[j] += forces_accum[:3] * dt
            L[j] += forces_accum[3:] * dt
            
            print('momentum ',j,':',p[j])
            print('angular momentum ',j,':',L[j])
            
            #compute auxiliary state
            v = p[j] / objects[j].mass
            
            R = quaternion_to_matrix(states[j][:4])
            Iinv = R @ objects[j].inertia_tensor_inv @ R.T
            omega = Iinv @ L[j]
            
            print('v',j,':',v)
            print('omega',j,':',omega)
                        
            qdot = np.array([0, omega[0], omega[1], omega[2]], np.float32)
            hamilton_product_host(qdot, states[j][:4], qdot)
            qdot *= 0.5
            
            #timestep update
            states[j][:4] += qdot * dt
            states[j][:4] /= la.norm(states[j][:4])
            states[j][4:7] += v * dt
            states[j][7:10] = omega
            states[j][10:13] = v
            states_d[j].copy_to_device(states[j])
              
        allstates.append([state.copy() for state in states])
        
    return allstates
            

## Testing
Move the slider to advance the time on these examples. You will also see a 3D visualization of the initial grid data structure to help understand the resolution the simulation is running at, with the two objects' occupied grid cells colored red and blue, respectively.

In [8]:
def joinmeshes(v1, f1, v2, f2):
    return np.vstack((v1, v2)), np.vstack((f1, f2 + v1.shape[0]))

In [9]:
def display_animation(v1, f1, v2, f2, states):
    fullv, fullf = joinmeshes(v1, f1, v2, f2)
    p = plot(fullv, fullf)
    n = len(states)
    @interact(t=ipywidgets.IntSlider(min=0, max=n-1, step=1))
    def ff(t):
        vs = []
        for j in range(2):
            v = (v1, v2)[j]
            R = quaternion_to_matrix(states[t][j][:4])
            x = states[t][j][4:7]
            newv = (R @ v.T).T + x
            vs.append(newv)
        fullv = np.vstack(vs)
        p.update_object(vertices=fullv)

### off-center collision

In [11]:
v, f = igl.read_triangle_mesh('mechanical/data/cylinder3.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0, 0.5, 2.1, 0,
                 0, 0, 0, 0, -10, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 0, 0.001, 1000)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 1.8771946430206299
points in obj1: 781952
points in obj2: 781952
copied data to GPU in 2.0380756855010986
initial momentum: [array([0., 0., 0.], dtype=float32), array([  0., -10.,   0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
frame 0
populated grid in 0.9926809692382812
computed individual point forces in 1.6039281005859376


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

reduced forces 0 on gpu in 0.27353033447265623
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0006853119730949402
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]
omega 1 : [0. 0. 0.]
frame 1
populated grid in 0.0021480000019073486
computed individual point forces in 0.0036915199756622314
reduced forces 0 on gpu in 0.0004403199851512909
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004413439929485321
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]
omega 1 : [0. 0. 0.]
frame 2
populated grid in 0.002089855909347534
computed individual point for

populated grid in 0.00208076810836792
computed individual point forces in 0.003761152029037476
reduced forces 0 on gpu in 0.00041574400663375855
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004410240054130554
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]
omega 1 : [0. 0. 0.]
frame 18
populated grid in 0.0021340160369873046
computed individual point forces in 0.0036125121116638184
reduced forces 0 on gpu in 0.000620415985584259
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0004412800073623657
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]

populated grid in 0.001957856059074402
computed individual point forces in 0.003169280052185059
reduced forces 0 on gpu in 0.00035737600922584536
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0003694080114364624
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]
omega 1 : [0. 0. 0.]
frame 35
populated grid in 0.0020285439491271972
computed individual point forces in 0.00300543999671936
reduced forces 0 on gpu in 0.000562175989151001
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00037347200512886046
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [  0. -10.   0.]
angular momentum  1 : [0. 0. 0.]
v 1 : [ 0.        -1.5985636  0.       ]

...

reduced forces 1 on gpu in 0.0003665919899940491
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07766583  0.01299771  0.68235085]
frame 919
populated grid in 0.0018889600038528443
computed individual point forces in 0.002542272090911865
reduced forces 0 on gpu in 0.00034508800506591797
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.09881917 -0.04566156  2.94153975]
reduced forces 1 on gpu in 0.0003604480028152466
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07766586  0.01

populated grid in 0.0018786879777908324
computed individual point forces in 0.002584575891494751
reduced forces 0 on gpu in 0.000568992018699646
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.0996559  -0.04629533  2.94150164]
reduced forces 1 on gpu in 0.0003543039858341217
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07766586  0.01287599  0.68235308]
frame 938
populated grid in 0.0018769919872283936
computed individual point forces in 0.002431999921798706
reduced forces 0 on gpu in 0.00033484798669815064
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.0

force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.10047589 -0.04685262  2.94146496]
reduced forces 1 on gpu in 0.0003758080005645752
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07766456  0.01276711  0.68235538]
frame 955
populated grid in 0.0019056639671325684
computed individual point forces in 0.0024391679763793943
reduced forces 0 on gpu in 0.0003338240087032318
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.10052497 -0.04688412  2.94146278]
reduced for

frame 973
populated grid in 0.0018411519527435302
computed individual point forces in 0.002513920068740845
reduced forces 0 on gpu in 0.00035123199224472044
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.10142396 -0.04742628  2.94142326]
reduced forces 1 on gpu in 0.0007219200134277344
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07766148  0.01264544  0.68235772]
frame 974
populated grid in 0.0018686399459838868
computed individual point forces in 0.0024330239295959474
reduced forces 0 on gpu in 0.00034201601147651673
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular moment

force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.10235032 -0.04792033  2.9413832 ]
reduced forces 1 on gpu in 0.001569375991821289
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [ 2.7292225  -2.2324603  -0.00396446]
angular momentum  1 : [-0.02744005  0.00515371  0.2735357 ]
v 1 : [ 0.43628356 -0.35687295 -0.00063374]
omega 1 : [-0.07765718  0.01253027  0.68236052]
frame 992
populated grid in 0.0018565119504928588
computed individual point forces in 0.0023982079029083252
reduced forces 0 on gpu in 0.00034505599737167357
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [-2.7292223e+00 -7.7675395e+00  3.9644931e-03]
angular momentum  0 : [ 0.03816588 -0.01031232  1.009125  ]
v 0 : [-4.3628353e-01 -1.2416905e+00  6.3374941e-04]
omega 0 : [ 0.10240252 -0.04794635  2.94138096]
reduced for

In [12]:
display_animation(v, f, v, f, states)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

### rotating in place

In [13]:
v, f = igl.read_triangle_mesh('mechanical/data/cylinder3.obj')
pose1 = np.array([1, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([1, 0, 0, 0, 0, 2.1, 0,
                 0, 10, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.02, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 100, 10, 0.001, 250)
display_animation(v, f, v, f, states)

minPt: [-1.1 -1.1 -1.1] maxPt: [1.1 3.1 1.1] gridLen: 0.02
initailized objects in 0.8670022487640381
points in obj1: 781952
points in obj2: 781952
copied data to GPU in 0.03277873992919922
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([ 0., 10.,  0.], dtype=float32)]
frame 0
populated grid in 0.0018862080574035644
computed individual point forces in 0.002564095973968506


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(55.5, 55.…

reduced forces 0 on gpu in 0.0006298879981040954
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00045977601408958433
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-5.45316198e-05  2.48289847e+01 -6.55960912e-05]
frame 1
populated grid in 0.0018433599472045899
computed individual point forces in 0.0027514879703521727
reduced forces 0 on gpu in 0.0006259520053863525
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0003543039858341217
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-5.58371475e-05  2.48289847e+01 -6.41437524e-05]
frame 2
populated grid in 0.0017448320388793945
c

force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00038089600205421447
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-7.20077402e-05  2.48289847e+01 -3.65973040e-05]
frame 19
populated grid in 0.0018821120262145995
computed individual point forces in 0.002711551904678345
reduced forces 0 on gpu in 0.0005269439816474915
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036137598752975463
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-7.25943922e-05  2.48289847e+01 -3.48645108e-05]
frame 20
populated grid in 0.0018862080574035644
computed individual point forces in 0.0027931840

computed individual point forces in 0.002857856035232544
reduced forces 0 on gpu in 0.0006092799901962281
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036351999640464784
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-7.64092319e-05  2.48289847e+01 -5.18074284e-06]
frame 37
populated grid in 0.002040832042694092
computed individual point forces in 0.002867199897766113
reduced forces 0 on gpu in 0.00033855998516082764
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036531201004981993
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [-7.62831621e-05  2.48289847e+01 -3.47146

...

reduced forces 1 on gpu in 0.00036934399604797365
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 5.67126675e-05  2.48289847e+01 -8.96918107e-06]
frame 197
populated grid in 0.002131968021392822
computed individual point forces in 0.0031477439403533934
reduced forces 0 on gpu in 0.00034201601147651673
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.0003580160140991211
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 5.65202265e-05  2.48289847e+01 -1.01152815e-05]
frame 198
populated grid in 0.002050112009048462
computed individual point forces in 0.003126271963119507
reduced forces 0 on gpu in 0.0003481599986553192
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular mome

computed individual point forces in 0.0029276158809661865
reduced forces 0 on gpu in 0.0006604800224304199
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036243200302124026
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 5.00131287e-05  2.48289847e+01 -2.87315375e-05]
frame 215
populated grid in 0.0020343360900878906
computed individual point forces in 0.002884608030319214
reduced forces 0 on gpu in 0.0003338240087032318
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00035942399501800537
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 4.94456351e-05  2.48289847e+01 -2.975

populated grid in 0.0018982080221176148
computed individual point forces in 0.003052544116973877
reduced forces 0 on gpu in 0.0003717119991779327
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036556801199913024
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 : [ 3.69064650e-05  2.48289847e+01 -4.53390732e-05]
frame 233
populated grid in 0.0019107840061187743
computed individual point forces in 0.002699199914932251
reduced forces 0 on gpu in 0.00045158401131629945
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00036556801199913024
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [ 0. 10.  0.]
v 1 : [0. 0. 0.]
omega 1 :

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=249), Output()), _dom_classes=('widget-interact'…

### Interlocking toruses

In [15]:
v, f = igl.read_triangle_mesh('mechanical/data/torus3.obj')
pose1 = np.array([math.sqrt(2)/2, 0, 0, math.sqrt(2)/2, 0, 0, 0,
                 0, 0, 0, 0, 0, 0], np.float32)
pose2 = np.array([math.sqrt(2)/2, math.sqrt(2)/2, 0, 0, 0, 1, 0,
                 0, 1, 0, 0, 0, 0], np.float32)
states = simulate_rigid_bodies(v, f, pose1, 1, v, f, pose2, 1, 0.04, np.array([-2.1, -2.1, -2.1]), np.array([2.1, 3.1, 2.1]), 100, 0, 0.001, 1000)
display_animation(v, f, v, f, states)

minPt: [-2.1 -2.1 -2.1] maxPt: [2.1 3.1 2.1] gridLen: 0.04
initailized objects in 0.045004844665527344
points in obj1: 18294
points in obj2: 18294
copied data to GPU in 0.03493523597717285
initial momentum: [array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32)]
initial angmom: [array([0., 0., 0.], dtype=float32), array([0., 1., 0.], dtype=float32)]
frame 0
populated grid in 0.0006973440051078796
computed individual point forces in 0.0006277120113372803


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(51.5, 52.…

reduced forces 0 on gpu in 0.0002536320090293884
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.000130048006772995
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.45451830e-03  4.61084048e+00 -1.15877420e-04]
frame 1
populated grid in 0.0006387839913368225
computed individual point forces in 0.0004689919948577881
reduced forces 0 on gpu in 0.0002229440063238144
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00011776000261306762
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.45231204e-03  4.61084130e+00 -1.24638041e-04]
frame 2
populated grid in 0.0005888000130653381
computed

frame 29
populated grid in 0.0005201600193977356
computed individual point forces in 0.0004935680031776428
reduced forces 0 on gpu in 0.0002528960108757019
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00010956799983978272
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.39324549e-03  4.61084250e+00 -3.69132557e-04]
frame 30
populated grid in 0.000587776005268097
computed individual point forces in 0.0004095999896526337
reduced forces 0 on gpu in 0.00021503999829292298
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00011776000261306762
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 

reduced forces 1 on gpu in 0.00014339199662208557
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.33646375e-03  4.61084089e+00 -6.18693615e-04]
frame 59
populated grid in 0.0006113280057907104
computed individual point forces in 0.00042681598663330076
reduced forces 0 on gpu in 0.00021987199783325196
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentum  0 : [0. 0. 0.]
v 0 : [0. 0. 0.]
omega 0 : [0. 0. 0.]
reduced forces 1 on gpu in 0.00011161600053310394
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [0. 0. 0.]
angular momentum  1 : [0. 1. 0.]
v 1 : [0. 0. 0.]
omega 1 : [ 2.33456479e-03  4.61084153e+00 -6.27230424e-04]
frame 60
populated grid in 0.0006010879874229431
computed individual point forces in 0.0004022400081157684
reduced forces 0 on gpu in 0.0002037760019302368
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [0. 0. 0.]
angular momentu

...

populated grid in 0.0007997440099716186
computed individual point forces in 0.0005754879713058471
reduced forces 0 on gpu in 0.0002576960027217865
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [ 0.12210318  0.53947216 -0.34507412]
angular momentum  0 : [-0.27374977  0.12212083 -0.10776271]
v 0 : [ 0.10428897  0.460766   -0.2947296 ]
omega 0 : [-0.52519621  0.19831095 -0.221077  ]
reduced forces 1 on gpu in 0.00011776000261306762
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [-0.12210321 -0.53947216  0.34507418]
angular momentum  1 : [-0.07256698  0.8862569  -0.001683  ]
v 1 : [-0.104289   -0.460766    0.29472965]
omega 1 : [-1.13239715  4.06932903  0.84612416]
frame 938
populated grid in 0.0006000639796257019
computed individual point forces in 0.0004207360148429871
reduced forces 0 on gpu in 0.00020160000026226045
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [ 0.12210318  0.53947216 -0.34507412]
angular momentum  0 : [-0.27374977  0.12212083 -0.

reduced forces 1 on gpu in 0.000143327996134758
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [-0.12210321 -0.53947216  0.34507418]
angular momentum  1 : [-0.07256698  0.8862569  -0.001683  ]
v 1 : [-0.104289   -0.460766    0.29472965]
omega 1 : [-1.08975909  4.07325509  0.99369159]
frame 966
populated grid in 0.0006072319746017456
computed individual point forces in 0.00044118401408195496
reduced forces 0 on gpu in 0.00022115199267864227
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [ 0.12210318  0.53947216 -0.34507412]
angular momentum  0 : [-0.27374977  0.12212083 -0.10776271]
v 0 : [ 0.10428897  0.460766   -0.2947296 ]
omega 0 : [-0.52539389  0.19809617 -0.22081839]
reduced forces 1 on gpu in 0.0001125119999051094
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [-0.12210321 -0.53947216  0.34507418]
angular momentum  1 : [-0.07256698  0.8862569  -0.001683  ]
v 1 : [-0.104289   -0.460766    0.29472965]
omega 1 : [-1.08789581  4.07342321  0.9991350

frame 994
populated grid in 0.0016988159418106078
computed individual point forces in 0.0004431680142879486
reduced forces 0 on gpu in 0.00021606400609016418
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [ 0.12210318  0.53947216 -0.34507412]
angular momentum  0 : [-0.27374977  0.12212083 -0.10776271]
v 0 : [ 0.10428897  0.460766   -0.2947296 ]
omega 0 : [-0.525586    0.19788955 -0.2205648 ]
reduced forces 1 on gpu in 0.00011238399893045425
force  1 : [0. 0. 0.]
torque  1 : [0. 0. 0.]
momentum  1 : [-0.12210321 -0.53947216  0.34507418]
angular momentum  1 : [-0.07256698  0.8862569  -0.001683  ]
v 1 : [-0.104289   -0.460766    0.29472965]
omega 1 : [-1.02502339  4.0790522   1.15584911]
frame 995
populated grid in 0.0005927680134773254
computed individual point forces in 0.0004266879856586456
reduced forces 0 on gpu in 0.00020070399343967437
force  0 : [0. 0. 0.]
torque  0 : [0. 0. 0.]
momentum  0 : [ 0.12210318  0.53947216 -0.34507412]
angular momentum  0 : [-0.27374977  0.1

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t', max=999), Output()), _dom_classes=('widget-interact'…

## Conclusions
Despite there being on the order of millions of points per object, this method can simulate a 1000 timestep simulation in just a couple of seconds on my RTX 2060S.

There are some drawbacks to the scatter-based approach to storing particles in the grid. Namely, there can potentially be race conditions such that some particles do not get recorded (due to being overwritten). A thread cannot know where there is a free space to store the particle index in the cell, so it scans for an empty space, leading to this problem. However, I have partitioned the cells into different regions for each object so that interactions between the objects are not missed as long as each has a particle in the vicinity. Furthermore, I only compute forces on particles that are actually in the data structure, because otherwise it would be possible for more particles to receive forces from the other object than were detected as neighbors in the other direction, breaking Newton's third law.